# Agentic RAG: Bauhaftpflicht-Fälle intelligent durchsuchen

## Überblick

In `4_rag` haben wir ein einfaches RAG-System gebaut:
- Frage rein → 1x Vektor-Suche → 5 Chunks → LLM antwortet

Das Problem: **Jede Frage wird gleich behandelt.** Egal ob du nach ähnlichen Fällen suchst, einen bestimmten Fall analysieren willst, oder eine Übersicht brauchst.

In `5_agentic_rag` geben wir einem **Agenten** mehrere **Tools** und lassen ihn **selbst entscheiden**, was er tut.

```
4_rag (fix):           Frage → Suche → Antwort
5_agentic_rag (flexibel): Frage → Agent denkt → Tool → Ergebnis → Agent denkt → evtl. nochmal Tool → Antwort
```

## 1. Setup

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## 2. Unsere Tools anschauen

Der Agent hat drei Tools zur Verfügung. Schauen wir uns an, was sie tun:

In [ ]:
from tools import vector_search, get_case_overview, list_cases

# Jedes Tool hat einen Namen und eine Beschreibung, die der Agent sieht:
for tool in [vector_search, get_case_overview, list_cases]:
    print(f"Tool: {tool.name}")
    print(f"Beschreibung: {tool.description[:100]}...")
    print()

## 3. Tools manuell testen

Bevor wir den Agent loslassen, testen wir die Tools einzeln:

### Tool 1: `list_cases` — Alle Fälle auflisten

In [ ]:
from tools import list_cases as _list_cases
from tools import get_case_overview as _get_overview
from tools import vector_search as _vector_search
from tools import _get_vectorstore, DATA_PATH
import json

# Alle Fälle auflisten
cases = []
for case_dir in sorted(DATA_PATH.iterdir()):
    if not case_dir.is_dir():
        continue
    bible_path = case_dir / "case_bible.json"
    if not bible_path.exists():
        continue
    with open(bible_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"- {data.get('case_id')}: {data.get('cluster')} | Status: {data.get('status')}")

print(f"\nTotal: {len(list(DATA_PATH.iterdir()))} Ordner")

### Tool 2: `get_case_overview` — Einen Fall im Detail

In [ ]:
# Case Bible von Fall W1 laden
bible_path = DATA_PATH / "W1" / "case_bible.json"
with open(bible_path, "r", encoding="utf-8") as f:
    case_data = json.load(f)

print(json.dumps(case_data, ensure_ascii=False, indent=2))

### Tool 3: `vector_search` — Semantische Suche

In [ ]:
# Vektor-Suche (nutzt die ChromaDB aus 4_rag)
vectorstore = _get_vectorstore()
results = vectorstore.similarity_search_with_score("Wasseraustritt", k=3)

for doc, score in results:
    meta = doc.metadata
    print(f"[{score:.3f}] Fall {meta.get('case_id')} | {meta.get('doc_typ')} | {meta.get('cluster')}")
    print(f"  {doc.page_content[:150]}...")
    print()

## 4. Den Agent erstellen und laufen lassen

Jetzt das Herzstück: Wir erstellen den Agent und lassen ihn Fragen beantworten.

Der Agent bekommt:
- **Instructions**: Wer er ist und wie er antworten soll
- **Tools**: Die drei Funktionen von oben

Dann entscheidet er **selbst**, welche Tools er für jede Frage braucht.

In [ ]:
from agents import Agent, Runner
from tools import vector_search, get_case_overview, list_cases

INSTRUCTIONS = """Du bist ein Experte für Bauhaftpflicht-Fälle in der Schweiz.
Du hilfst Sachbearbeitern, ihre Fälle zu analysieren und vergleichbare Fälle zu finden.

Wichtige Regeln:
1. Nutze IMMER deine Tools, um Informationen zu finden — erfinde nichts.
2. Wenn du mehrere Fälle vergleichen sollst, hole jeden Fall einzeln.
3. Gib immer die Fall-ID(s) an, aus denen die Information stammt.
4. Antworte auf Deutsch.
5. Wenn du Beträge nennst, nutze das Format 'CHF 50'000'.
6. Wenn du nicht genug Information findest, sage das klar.
"""

agent = Agent(
    name="Bauhaftpflicht Agent",
    instructions=INSTRUCTIONS,
    tools=[vector_search, get_case_overview, list_cases],
)

### Frage 1: Einfache Suche

Diese Frage könnte auch `4_rag` beantworten. Der Agent wird `vector_search` nutzen.

In [ ]:
result = await Runner.run(agent, "Gibt es Fälle mit Wasserabdichtungsproblemen?")
print(result.final_output)

### Frage 2: Übersichtsfrage

Hier wird der Agent `list_cases` nutzen — das konnte `4_rag` nicht.

In [ ]:
result = await Runner.run(agent, "Wie viele Fälle gibt es und welche Cluster kommen vor?")
print(result.final_output)

### Frage 3: Spezifischer Fall

Der Agent wird `get_case_overview` aufrufen — gezielt statt zufällige Chunks.

In [ ]:
result = await Runner.run(agent, "Was sind die wichtigsten Fakten zu Fall W3?")
print(result.final_output)

### Frage 4: Vergleich (mehrere Tool-Aufrufe)

Das ist die Stärke des Agenten: Er wird **mehrere Tools nacheinander** aufrufen, um beide Fälle zu holen und zu vergleichen.

In [ ]:
result = await Runner.run(agent, "Vergleiche Fall W1 mit Fall H2. Was sind die Unterschiede?")
print(result.final_output)

### Frage 5: Kombinierte Frage

Hier muss der Agent zuerst suchen und dann Details nachladen:

In [ ]:
result = await Runner.run(
    agent,
    "Welche Fälle im Cluster Wasser wurden per Vergleich gelöst und wie hoch waren die Beträge?"
)
print(result.final_output)

## 5. Interaktiver Chat-Modus

Du kannst den Agent auch im interaktiven Modus nutzen (wie Shaws YouTube-Agent).
Tippe deine Fragen ein und der Agent antwortet. Beende mit `exit`.

In [ ]:
from agents import run_demo_loop

await run_demo_loop(agent)

## 6. Warum ist das besser als einfaches RAG?

| Frage | 4_rag | 5_agentic_rag |
|-------|-------|---------------|
| "Gibt es ähnliche Wasserschäden?" | 1x Suche, OK | 1x vector_search, OK |
| "Was ist der Stand von Fall W3?" | 5 zufällige Chunks | get_case_overview("W3") — gezielt |
| "Vergleiche W1 mit H2" | Kann nur einen suchen | Holt beide Fälle, vergleicht |
| "Wie viele Fälle gibt es?" | Versagt komplett | list_cases() |
| "Welche Wasserschäden gab es und wie hoch?" | 5 Chunks, unvollständig | list_cases + get_case_overview |

Der entscheidende Unterschied: **Der Agent entscheidet bei jeder Frage neu, welche Strategie er verwendet.**